In [2]:
from tangelo import SecondQuantizedMolecule
from tangelo.algorithms import VQESolver, FCISolver, CCSDSolver, BuiltInAnsatze
from tangelo.toolboxes.ansatz_generator.ansatz import Ansatz

In [3]:
def vqe_simulate(second_quantized_mol):
    vqe_options = {"molecule": second_quantized_mol}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    return vqe_solver.simulate()

In [71]:
def vqe_resources(second_quantized_mol):
    vqe_options = {"molecule": second_quantized_mol}
    vqe_solver = VQESolver(vqe_options)
    vqe_solver.build()
    return vqe_solver.get_resources()

In [5]:
def simulate_classically(second_quantized_mol):

    # Uncomment the following lines if FCI simulation is feasible
    # fci_solver = FCISolver(second_quantized_mol)
    # energy_fci = fci_solver.simulate()
    # print(f"FCI energy: \t {energy_fci}")

    ccsd_solver = CCSDSolver(second_quantized_mol)
    energy_ccsd = ccsd_solver.simulate()
    print(f"CCSD energy: \t {energy_ccsd}")

In [70]:
def vqe_resources_with_mappings(molecule, ansatz):
 
    qubit_mappings = ['jw', 'bk', 'scbk']  # List of different qubit mappings
    resources_list = []

    for qm in qubit_mappings:
        # Define VQE options with the given molecule, ansatz, and current qubit mapping
        vqe_options = {"molecule": molecule, "ansatz": ansatz, "qubit_mapping": qm}
        vqe_solver = VQESolver(vqe_options)
        vqe_solver.build()
  

        # Get resources and store in the list
        resources = vqe_solver.get_resources()
        resources_list.append({"qubit_mapping": qm, "resources": resources})

    return resources_list

In [69]:
def vqe_resources_with_ansatze(molecule, ansatze_list):

    resources_list = []

    for az in ansatze_list:
        # Define VQE options with the given molecule and ansatz
        vqe_options = {"molecule": molecule, "ansatz": az, "up_then_down": True}
        vqe_solver = VQESolver(vqe_options)
        vqe_solver.build()


        # Get resources and store in the list
        resources = vqe_solver.get_resources()
        resources_list.append({"ansatz": az, "resources": resources})

    return resources_list

In [120]:
def print_orbital_energies(molecule):

    orbital_energies = molecule.mo_energies
    occupations = molecule.mo_occ

    orbital_info = []
    for idx, (energy, occ) in enumerate(zip(orbital_energies, occupations)):
        orbital_info.append((idx, energy, occ))

    # Display orbital information
    print(f"Number of active electrons: {molecule.n_active_electrons}\n")
    print("Orbital Index | Occupation Number | Orbital Energy (Hartree)")
    print("------------------------------------------------------------")
    for idx, energy, occ in orbital_info:
        print(f"{idx:<13} {occ:<18} {energy}")
    

In [167]:
def KMeans1D_DP(data, K):
    """
    Performs optimal 1D k-means clustering via dynamic programming.
    
    Parameters:
    - data: 1D numpy array
    - K: number of clusters
    
    Returns:
    - labels: numpy array of cluster labels corresponding to the original data order
    """
    import numpy as np
    
    n = len(data)
    # Sort data and keep track of original indices
    sorted_indices = np.argsort(data)
    sorted_data = data[sorted_indices]
    
    # Precompute cumulative sums and cumulative squared sums
    data_cumsum = np.zeros(n+1)
    data_sq_cumsum = np.zeros(n+1)
    for i in range(1, n+1):
        data_cumsum[i] = data_cumsum[i-1] + sorted_data[i-1]
        data_sq_cumsum[i] = data_sq_cumsum[i-1] + sorted_data[i-1]**2
    
    # Initialize the cost and segmentation arrays
    cost = np.full((n+1, K+1), np.inf)
    backtrack = np.zeros((n+1, K+1), dtype=int)
    
    cost[0][0] = 0  # Base case
    
    # Dynamic programming to compute optimal cost
    for i in range(1, n+1):
        for k in range(1, min(i, K)+1):
            for j in range(k-1, i):
                count = i - j
                sum_x = data_cumsum[i] - data_cumsum[j]
                sum_x_sq = data_sq_cumsum[i] - data_sq_cumsum[j]
                mean = sum_x / count
                ssd = sum_x_sq - 2 * mean * sum_x + count * mean**2
                total_cost = cost[j][k-1] + ssd
                if total_cost < cost[i][k]:
                    cost[i][k] = total_cost
                    backtrack[i][k] = j
    
    # Backtracking to assign labels
    labels = np.zeros(n, dtype=int)
    k = K
    i = n
    while k > 0:
        j = backtrack[i][k]
        labels[j:i] = k-1
        i = j
        k -=1
    
    # Map labels back to original data order
    original_labels = np.zeros(n, dtype=int)
    original_labels[sorted_indices] = labels
    
    return original_labels


In [168]:
import numpy as np

def generate_freeze_lists(molecule, k_core, k_virtual):
    """
    Generates multiple freeze lists by clustering core (occupied) and virtual orbitals
    based on their energies using optimal 1D clustering via dynamic programming.

    Parameters:
    - molecule: SecondQuantizedMolecule object representing the molecule.
    - k_core: Number of clusters to form among core (occupied) orbitals.
    - k_virtual: Number of clusters to form among virtual orbitals.

    Returns:
    - freeze_lists: A list of freeze lists (each is a list of orbital indices to freeze).
    """
    orbital_energies = molecule.mo_energies
    occupations = molecule.mo_occ

    orbital_info = []
    for idx, (energy, occ) in enumerate(zip(orbital_energies, occupations)):
        orbital_info.append((idx, energy, occ))

    print(f"Number of active electrons: {molecule.n_active_electrons}")

    # Collect core orbitals (occupation == 2.0)
    core_indices = []
    core_energies = []
    for idx, energy, occ in orbital_info:
        if occ == 2.0:
            core_indices.append(idx)
            core_energies.append(energy)

    core_indices = np.array(core_indices)
    core_energies = np.array(core_energies)

    # Cluster core orbitals using dynamic programming
    if len(core_indices) == 0:
        print("No core orbitals to cluster.")
        core_clusters = {}
        core_cluster_order = []
    else:
        k_core = min(k_core, len(core_indices))
        core_labels = KMeans1D_DP(core_energies, k_core)
        # Group core orbitals by clusters
        core_clusters = {}
        for label in range(k_core):
            cluster_indices = core_indices[core_labels == label]
            core_clusters[label] = (cluster_indices)
        core_cluster_order = (range(k_core))
        # Print cluster info
        print(f"\nClustered core orbitals into {k_core} groups:")
        for label in core_cluster_order:
            indices = core_clusters[label]
            energies = [orbital_energies[i] for i in indices]
            print(f"Core Cluster {label+1}: Orbitals {indices}, Energies {energies}")

    # Collect virtual orbitals (occupation == 0.0)
    virtual_indices = []
    virtual_energies = []
    for idx, energy, occ in orbital_info:
        if occ == 0.0:
            virtual_indices.append(idx)
            virtual_energies.append(energy)

    virtual_indices = np.array(virtual_indices)
    virtual_energies = np.array(virtual_energies)

    # Cluster virtual orbitals using dynamic programming
    if len(virtual_indices) == 0:
        print("No virtual orbitals to cluster.")
        virtual_clusters = {}
        virtual_cluster_order = []
    else:
        k_virtual = min(k_virtual, len(virtual_indices))
        virtual_labels = KMeans1D_DP(virtual_energies, k_virtual)
        # Group virtual orbitals by clusters
        virtual_clusters = {}
        for label in range(k_virtual):
            cluster_indices = virtual_indices[virtual_labels == label]
            virtual_clusters[label] = (cluster_indices)
        virtual_cluster_order = (range(k_virtual))
        # Print cluster info
        print(f"\nClustered virtual orbitals into {k_virtual} groups:")
        for label in virtual_cluster_order:
            indices = virtual_clusters[label]
            energies = [orbital_energies[i] for i in indices]
            print(f"Virtual Cluster {label+1}: Orbitals {indices}, Energies {energies}")

    # Generate combinations according to the specified logic
    freeze_lists = []
    num_core_clusters = len(core_cluster_order)
    num_virtual_clusters = len(virtual_cluster_order)

    for c in range(1, num_core_clusters + 1):
        # Core clusters to include
        core_clusters_to_include = core_cluster_order[:c]
        core_orbitals_to_freeze = []
        for label in core_clusters_to_include:
            core_orbitals_to_freeze.extend(core_clusters[label])
        for v in range(0, num_virtual_clusters + 1):
            # Virtual clusters to include
            virtual_clusters_to_include = virtual_cluster_order[-v:] if v > 0 else []
            virtual_orbitals_to_freeze = []
            for label in virtual_clusters_to_include:
                virtual_orbitals_to_freeze.extend(virtual_clusters[label])
            freeze_list = core_orbitals_to_freeze + virtual_orbitals_to_freeze
            freeze_lists.append(freeze_list)
            print(f"\nGenerated freeze list with core clusters {core_clusters_to_include} and virtual clusters {virtual_clusters_to_include}")
            print(f"Freeze list orbitals: {freeze_list}")

    return freeze_lists



In [233]:
# Define the LiH molecule geometry
LiH = [('Li', (0, 0, 0)), ('H', (0, 0, 1.5949))]

# Create a SecondQuantizedMolecule instance for LiH
mol_LiH = SecondQuantizedMolecule(LiH, q=0, spin=0, basis="sto-3g")

# Define the Cr element
Cr= [['Cr', (0, 0, 0)]]

# Create a SecondQuantizedMolecule instance for Cr_VI
Cr_VI = SecondQuantizedMolecule(Cr, q=6, spin=0, basis="LANL2DZ", frozen_orbitals=None)

# Create a SecondQuantizedMolecule instance for Cr_III
Cr_III = SecondQuantizedMolecule(Cr, q=3, spin=3, basis="LANL2DZ", frozen_orbitals=None)

In [234]:
def print_molecule_properties(name, molecule):
    print(f"=== {name} Properties ===")
    print(f"Charge (q): {molecule.q}")
    print(f"Spin (2S): {molecule.spin}")
    print(f"Number of Electrons: {molecule.n_electrons}")
    print(f"Number of Active Electrons: {molecule.n_active_electrons}")
    print(f"Basis Set: {molecule.basis}")
    print(f"Frozen Orbitals: {molecule.frozen_orbitals}")
    print("--------------------------\n")

In [235]:
print_molecule_properties("LiH", mol_LiH)
print_molecule_properties("Cr_III", Cr_III)
print_molecule_properties("Cr_VI", Cr_VI)

=== LiH Properties ===
Charge (q): 0
Spin (2S): 0
Number of Electrons: 4
Number of Active Electrons: 2
Basis Set: sto-3g
Frozen Orbitals: frozen_core
--------------------------

=== Cr_III Properties ===
Charge (q): 3
Spin (2S): 3
Number of Electrons: 21
Number of Active Electrons: 21
Basis Set: LANL2DZ
Frozen Orbitals: None
--------------------------

=== Cr_VI Properties ===
Charge (q): 6
Spin (2S): 0
Number of Electrons: 18
Number of Active Electrons: 18
Basis Set: LANL2DZ
Frozen Orbitals: None
--------------------------


In [239]:
print_orbital_energies(Cr_III)

Number of active electrons: 21

Orbital Index | Occupation Number | Orbital Energy (Hartree)
------------------------------------------------------------
0             2.0                -12.450388168642943
1             2.0                -9.656067875462
2             2.0                -9.65606624122188
3             2.0                -9.656064608113377
4             2.0                -6.4812279437245
5             2.0                -6.481226829404323
6             2.0                -6.481225704351173
7             2.0                -6.479526799840919
8             2.0                -6.4795267916045
9             1.0                -1.880148215051068
10            1.0                -0.7975336380039709
11            1.0                -0.7975336190404309
12            0.0                -0.7535824185769827
13            0.0                -0.7535812790780597
14            0.0                -0.7535801148644445
15            0.0                -0.7031960307938059
16            0

In [238]:
freeze_lists = generate_freeze_lists(Cr_III, 4, 4)


Number of active electrons: 21

Clustered core orbitals into 4 groups:
Core Cluster 1: Orbitals [0], Energies [-12.450388168642943]
Core Cluster 2: Orbitals [1 2 3], Energies [-9.656067875462, -9.65606624122188, -9.656064608113377]
Core Cluster 3: Orbitals [4 5 6], Energies [-6.4812279437245, -6.481226829404323, -6.481225704351173]
Core Cluster 4: Orbitals [7 8], Energies [-6.479526799840919, -6.4795267916045]

Clustered virtual orbitals into 4 groups:
Virtual Cluster 1: Orbitals [12 13 14], Energies [-0.7535824185769827, -0.7535812790780597, -0.7535801148644445]
Virtual Cluster 2: Orbitals [15], Energies [-0.7031960307938059]
Virtual Cluster 3: Orbitals [16 17 18], Energies [-0.6596666411582076, -0.6596635891836166, -0.659660494598583]
Virtual Cluster 4: Orbitals [19 20 21], Energies [-0.3052729883268533, -0.3052724376345083, -0.30527189127184107]

Generated freeze list with core clusters range(0, 1) and virtual clusters []
Freeze list orbitals: [0]

Generated freeze list with core cl

In [218]:
import multiprocessing
import time

def build_with_timeout(molecule_params, timeout=60):
    

In [240]:
"""Important: Sort list and then limit to 1 minute
   Also maybe don't cluster the core elctrons?"""
frozen_molecules_dict = {}

for idx, freeze_list in enumerate(freeze_lists, start=1):
    int_freeze_list = [int(x) for x in freeze_list]

    
    freeze_list_key = tuple(int_freeze_list)
    
    # Create a new SecondQuantizedMolecule with frozen orbitals
    try:
        Cr_III_custom_frozen = SecondQuantizedMolecule(
            Cr,
            q=3,
            spin=3,
            basis="LANL2DZ",
            frozen_orbitals=int_freeze_list
        )
        
        if len(int_freeze_list) > 8:
            print(f"\nCreating frozen molecule for Freeze List {idx}: {int_freeze_list}")
            resources = vqe_resources(Cr_III_custom_frozen)
            print(resources)
            if resources["circuit_width"] < 15:
                vqe_energy = vqe_simulate(Cr_III_custom_frozen)
                print(f"vqe_energy = {vqe_energy}")
    except TypeError as e:
        print(f"TypeError for Freeze List {idx}: {e}")
    except Exception as e:
        print(f"Unexpected error for Freeze List {idx}: {e}")


Creating frozen molecule for Freeze List 5: [0, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
{'qubit_hamiltonian_terms': 7966, 'circuit_width': 22, 'circuit_depth': 105, 'circuit_2qubit_gates': 80, 'circuit_var_gates': 8, 'vqe_variational_parameters': 108}

Creating frozen molecule for Freeze List 8: [0, 1, 2, 3, 16, 17, 18, 19, 20, 21]
{'qubit_hamiltonian_terms': 10333, 'circuit_width': 24, 'circuit_depth': 70344, 'circuit_2qubit_gates': 61488, 'circuit_var_gates': 3624, 'vqe_variational_parameters': 1565}

Creating frozen molecule for Freeze List 9: [0, 1, 2, 3, 15, 16, 17, 18, 19, 20, 21]
{'qubit_hamiltonian_terms': 7950, 'circuit_width': 22, 'circuit_depth': 64771, 'circuit_2qubit_gates': 55456, 'circuit_var_gates': 3408, 'vqe_variational_parameters': 1008}

Creating frozen molecule for Freeze List 10: [0, 1, 2, 3, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]
{'qubit_hamiltonian_terms': 3777, 'circuit_width': 16, 'circuit_depth': 105, 'circuit_2qubit_gates': 80, 'circuit_var_gates': 8, 'vqe

In [166]:

    # Perform simulations or further processing with 'frozen_molecule'


Number of active electrons: 21

Clustered core orbitals into 3 groups:
Core Cluster 1: Orbitals [0], Energies [-12.450388167225722]
Core Cluster 2: Orbitals [1 2 3], Energies [-9.65606624144028, -9.656066240108055, -9.656066238814043]
Core Cluster 3: Orbitals [4 5 6 7 8], Energies [-6.481226824872179, -6.481226824155213, -6.481226823078253, -6.479526794447777, -6.479526794297061]

Clustered virtual orbitals into 3 groups:
Virtual Cluster 1: Orbitals [12 13 14], Energies [-0.7535812710254618, -0.7535812703857644, -0.753581269194651]
Virtual Cluster 2: Orbitals [15 16 17 18], Energies [-0.7031960313467708, -0.6596635758608538, -0.659663573875369, -0.6596635709654455]
Virtual Cluster 3: Orbitals [19 20 21], Energies [-0.30527243942537624, -0.3052724388892152, -0.3052724385369162]

Generated freeze list with core clusters range(0, 1) and virtual clusters []
Freeze list orbitals: [0]

Generated freeze list with core clusters range(0, 1) and virtual clusters range(2, 3)
Freeze list orbitals:

[[0],
 [0, 19, 20, 21],
 [0, 15, 16, 17, 18, 19, 20, 21],
 [0, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
 [0, 1, 2, 3],
 [0, 1, 2, 3, 19, 20, 21],
 [0, 1, 2, 3, 15, 16, 17, 18, 19, 20, 21],
 [0, 1, 2, 3, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21],
 [0, 1, 2, 3, 4, 5, 6, 7, 8],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 19, 20, 21],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 15, 16, 17, 18, 19, 20, 21],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]]